In [1]:
import numpy as np
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
import datetime
from scipy.interpolate import griddata
import tqdm
from regrid import regrid
import matplotlib.pyplot as plt
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


### depth

In [5]:
#import lon, lat & snod values from original SM dataset
datapath = '/Users/carmennab/Dropbox/OI_PolarSnow/data/SnowModelLG/'
file = datapath+'SM_snod_ERA5_01Aug1980-31Jul2018_v01.nc'
data = Dataset(file,'r')

lon = data['x']
lat = data['y']
snod = data['snod'][10745:] # 1/1/2010 onwards only
snod_orig = np.array(snod) ; lon = np.array(lon) ; lat = np.array(lat)
snod_orig[snod_orig==-9999] = np.nan

#import lon, lat & snod values from SM extension
datapath = '/Users/carmennab/Dropbox/OI_PolarSnow/data/SnowModelLG/'
file = datapath+'SM_snod_ERA5_ease_01Aug2018-31Jul2021.nc'
data = Dataset(file,'r')
snod_ext = data['snod'][:884]
snod_ext = np.array(snod_ext)
snod_ext[snod_ext==-9999] = np.nan

# combine SM_orig and SM_ext
snow = np.concatenate((snod_orig,snod_ext),axis=0)

In [6]:
# regrid onto EASE grid and save
mymask = Dataset('/Users/carmennab/Dropbox/OI_PolarSnow/data/SnowModelLG/mask.nc')
SM_lons = np.array(mymask['lon'])
SM_lats = np.array(mymask['lat'])
EASE_lats = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lat_25km.npy')
EASE_lons = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lon_25km.npy')
days = np.arange(0,4018)
snows = np.full((4018,360,360), np.nan)
for day in tqdm.tqdm(days):
    snod = snow[day]
    SM_new = regrid(snod, SM_lons, SM_lats, EASE_lons, EASE_lats, method='linear')
    snows[day] = SM_new

100%|█████████████████████████████████████| 4018/4018 [1:21:32<00:00,  1.22s/it]


In [7]:
# create SM-LG NETCDF
data_vars = {'Snow Depth':(['t','x','y'],snows)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'Longitude':(['x','y'],EASE_lons),
                        'Latitude':(['x','y'],EASE_lats),
                         'Day':(['t'],days)}     )
ds.to_netcdf(f'/Users/carmennab/Dropbox/OI_PolarSnow/EASE/data/SMLG_snod_EASE.nc','w')